In [1]:
# 🧠 코드 구성 요약
# | 단계         | 설명                                            |
# | ---------- | --------------------------------------------- |
# | 1️⃣ 파일 로딩  | 데이터 타입 최적화 적용 (`category`, `int32`, `string`) |
# | 2️⃣ 전처리    | `이용금액`, `이용건수` 정규화 (`StandardScaler`)         |
# | 3️⃣ 이상값 탐지 | `Isolation Forest` (이상소비 탐지용)                 |
# | 4️⃣ 군집 분석  | `KMeans` (유사 소비패턴 분류용)                        |
# | 5️⃣ 결과 저장  | `is_outlier`, `cluster` 컬럼 포함하여 CSV로 저장       |

In [2]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 파일 경로
file_path = r"D:\따릉이\공공자전거23년\카드소비데이터.csv"
output_path = r"D:\따릉이\공공자전거23년\카드소비데이터_분석결과.csv"

# 데이터 타입 최적화 설정
dtype_dict = {
    '성별': 'category',
    '시간대별': 'category',
    '시군구': 'category',
    '시도': 'category',
    '업종명': 'category',
    '연령대': 'category',
    '요일': 'category',
    '월별_날짜': 'string',  # 날짜로 연산할 경우 'datetime64[ns]'도 가능
    '이용건수': 'int16',
    '이용금액': 'int32'
}

# CSV 읽기
df = pd.read_csv(file_path, dtype=dtype_dict, encoding='utf-8')

# 분석 대상 수치형 변수 선택
features = ['이용금액', '이용건수']
X = df[features]

# 스케일링 (정규화)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Isolation Forest로 이상값 탐지
iso_forest = IsolationForest(contamination=0.03, random_state=42)
df['is_outlier'] = iso_forest.fit_predict(X_scaled)
df['is_outlier'] = df['is_outlier'].apply(lambda x: 1 if x == -1 else 0)

# KMeans 클러스터링 (4개 군집)
kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# 결과 저장
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print("🎉 분석 결과 저장 완료:", output_path)


🎉 분석 결과 저장 완료: D:\따릉이\공공자전거23년\카드소비데이터_분석결과.csv
